[View in Colaboratory](https://colab.research.google.com/github/aeevrubin/DS_questions/blob/master/Fraudulent_retail_accounts.ipynb)

# problem statement
Question 1 - Fradulent retail accounts
Below is a daily table for an active accounts at Shopify (an online ecommerce, retail platform).



The table is called store_account and the columns are:

*  Column Name	| Data Type	| Description
*    store_id	|    integer	|    a unique Shopify store id
*    date        |    	string    |    	date
*    status     |    	string   |    	Possible values are: [‘open’, 'closed’, ‘fraud’]
*    revenue   |    	double  |    	Amount of spend in USDList item

Here's some more information about the table:

The granularity of the table is store_id and day
Assume “close” and “fraud” are permanent labels
Active = daily revenue > 0
Accounts get labeled by Shopify as fraudulent and they no longer can sell product
Every day of the table has every store_id that has ever been used by Shopify

Given the above, what percent of active stores were fraudulent by day? 

In [0]:
# imports
import pandas as pd
import numpy as np
import datetime

In [0]:
# assumptions: there is activity every day, there are no null values
data_dict = {'store_id':[1254231,154321,125543,  1254231,154321,125543,  1254231,154321,125543], 
             'date': ['May 12, 2018','May 12, 2018','May 12, 2018',           
                      'May 11, 2018','May 11, 2018','May 11, 2018',
                      'May 10, 2018','May 10, 2018','May 10, 2018'],
            'status': ['closed','fraud','closed', 'open','fraud','closed', 'open','open','closed' ], 
             'revenue': [4000.32, 500.08, 1600,  800.1, 500.,1,              1,2,3]}
store_account = pd.DataFrame.from_dict(data_dict)
new_entry={'store_id':1111, 
           'date':'May 10, 2018', 
           'status': 'open',
            'revenue': 10.0}
store_account.tail(len(store_account))

,date,revenue,status,store_id
0,"May 12, 2018",4000.32,closed,1254231
1,"May 12, 2018",500.08,fraud,154321
2,"May 12, 2018",1600.00,closed,125543
3,"May 11, 2018",800.10,open,1254231
4,"May 11, 2018",500.00,fraud,154321
5,"May 11, 2018",1.00,closed,125543
6,"May 10, 2018",1.00,open,1254231
7,"May 10, 2018",2.00,open,154321
8,"May 10, 2018",3.00,closed,125543


In [0]:
#unique day shouldn't have unique id, id's should repeat daily
store_account.groupby(['date','status'])['store_id'].unique()

date          status
May 10, 2018  closed             [125543]
              open      [1254231, 154321]
May 11, 2018  closed             [125543]
              fraud              [154321]
              open              [1254231]
May 12, 2018  closed    [1254231, 125543]
              fraud              [154321]
Name: store_id, dtype: object

In [0]:
#no status!=closed for may 11 is a problem
#what percent of active stores were fraudulent by day?

n_active_stores_per_day = store_account[store_account.revenue>0].groupby(['date'])['store_id'].nunique()
n_fraud_active_stores_per_day = store_account[(store_account.revenue>0) & (store_account.status=='fraud')].groupby(['date'])['store_id'].nunique()
                                        
df = pd.concat([n_active_stores_per_day,n_fraud_active_stores_per_day],join='outer',axis=1).fillna(0)
df.columns = ['n_active_stores_per_day','n_fraud_active_stores_per_day']
df['percentage'] = df.n_fraud_active_stores_per_day/df.n_active_stores_per_day*100

df

,n_active_stores_per_day,n_fraud_active_stores_per_day,percentage
"May 10, 2018",3,0.0,0.000000
"May 11, 2018",3,1.0,33.333333
"May 12, 2018",3,1.0,33.333333


In [0]:
#pivot table to understand better and force entries for all date-store combos

id_vs_date = store_account.pivot(index='store_id',columns='date',values='status')
id_vs_date

date,"May 10, 2018","May 11, 2018","May 12, 2018"
store_id,,,
125543,closed,closed,closed
154321,open,fraud,fraud
1254231,open,open,closed


In [0]:
id_vs_date.iloc[:,0].value_counts()

open      2
closed    1
Name: May 10, 2018, dtype: int64

In [0]:
#redo with pivot_table
pt = pd.pivot_table(store_account, index=['date'], columns=['status'], values=['store_id'],aggfunc='count',
                    margins=True,margins_name='Sum').fillna(0)
pt['percentage'] = pt.store_id.fraud/pt.store_id.Sum*100
#print(pt.info())
print(pt)

             store_id                percentage
status         closed fraud open Sum           
date                                           
May 10, 2018      1.0   0.0  2.0   3   0.000000
May 11, 2018      1.0   1.0  1.0   3  33.333333
May 12, 2018      2.0   1.0  0.0   3  33.333333
Sum               4.0   2.0  3.0   9  22.222222


In [0]:
# change date to datetime object as the index, but keep the data column as strings
#store_account = store_account.set_index(pd.to_datetime(store_account.date))
#could try with resample too I guess